In [33]:
from dotenv import load_dotenv 
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient
from msgraph.generated.users.item.messages.messages_request_builder import MessagesRequestBuilder
from msgraph.generated.search.query.query_post_request_body import QueryPostRequestBody
from msgraph.generated.search.query.query_request_builder import QueryRequestBuilder
from msgraph.generated.search.search_request_builder import SearchRequestBuilder
from msgraph.generated.models.search_request import SearchRequest
from msgraph.generated.models.entity_type import EntityType
from msgraph.generated.models.search_query import SearchQuery
from msgraph.generated.models.search_response import SearchResponse
import os
import json
import asyncio 
import uuid

In [37]:
from typing import Tuple

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding

In [45]:
kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
api_key, org_id = sk.openai_settings_from_dot_env()
kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))
kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x118e0ba10>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x118e0b8d0>}

In [47]:
print(org_id)

org-wYMNsr9TXbJmVu2K24SB1xbk


In [2]:
load_dotenv()

True

In [3]:
TENANT_ID = os.getenv("TENANT_ID")

In [4]:
CLIENT_ID = os.getenv("CLIENT_ID")

In [5]:
graphUserScopes = os.getenv("graphUserScopes").split(' ')

In [6]:
print(CLIENT_ID)

48943d42-a075-43c5-8d93-3f2da879c1be


In [7]:
fabs = 'fabian@fabster.onmicrosoft.com'

In [8]:
adele = 'adelev@fabster.onmicrosoft.com'

In [9]:
## ensure that you have turned on DefaultClientType to Yes in the App Registration

In [10]:
print(graphUserScopes)

['User.Read', 'Mail.ReadWrite', 'Chat.ReadWrite', 'Calendars.Read', 'Files.Read.All', 'Sites.Read.All']


In [11]:
credential = DeviceCodeCredential(client_id=CLIENT_ID, tenant_id=TENANT_ID) 
scopes = graphUserScopes
client = GraphServiceClient(credentials=credential, scopes=scopes)

In [13]:
result = await client.me.get()

In [14]:
print(result.display_name)

Adele Vance


In [ ]:
async def get_user(currUser):
    result = await client.users.by_user_id(currUser).get()
    if result:
        print(result.display_name)

In [ ]:
await (get_user('adelev@fabster.onmicrosoft.com'))

In [ ]:
async def get_user_messages(currUser):
    messages = await (client.users.by_user_id(currUser).messages.get())
    if messages and messages.value:
        for msg in messages.value:
            print(msg.subject, msg.id, msg.from_, msg.body_preview)

In [ ]:
await get_user_messages(adele)

In [ ]:
async def get_user_chats(currUser):
    chats = await (client.users.by_user_id(currUser).chats.get())
    if chats and chats.value:
        for cht in chats.value:
            print(cht.topic)

In [ ]:
await get_user_chats(adele)

## Work with Teams(Chat) Search Query Endpoint

In [15]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.ChatMessage
			],
			query = SearchQuery(
				query_string = "Adele",
			),
		),
	],
)

chat_result = await client.search.query.post(request_body)

In [16]:
chat_data = chat_result.value

In [18]:
#chat_data

In [20]:
search_response = chat_data[0]  # Assuming there's only one SearchResponse object in the list

# Initialize an empty list to store summaries and chat IDs
summaries_and_chat_ids = []

# Iterate over hits_containers in the SearchResponse object
for hits_container in search_response.hits_containers:
    # Iterate over each hit in the container
    for hit in hits_container.hits:
        # Extract the summary
        summary = hit.summary

        # Extract the chatId from additional_data
        chat_id = hit.resource.additional_data.get('chatId', 'Unknown')

        # Append the chatId and summary as a tuple to the list
        summaries_and_chat_ids.append((chat_id, summary))

# Now 'summaries_and_chat_ids' contains the chatId and summary of each search hit
for item in summaries_and_chat_ids:
    print(item)

('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'hmmm which one? the one closer to the book store by Rokin near the Yo boba store?')
('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'hey Im in Amsterdam downtown lets get some Fries from that spot near Amsterdam Central next to the Uni Clo store')
('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'I need to fix Kaleb Williams bicycle today What tools should i bring based on this document here https://fabster-my.sharepoint.com/:w:/g/personal/fabian_fabster_onmicrosoft_com/EfYFSS6IwSZNnyCyYPcK4IkBRiFeuXhA3cHn74Ha6Ag_Ow https://fabster-my.sharepoint.c')
('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'how are you...')
('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'hi adele long time')
('19:28031fbd-75af-4

In [22]:
print(summaries_and_chat_ids)

[('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'hmmm which one? the one closer to the book store by Rokin near the Yo boba store?'), ('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'hey Im in Amsterdam downtown lets get some Fries from that spot near Amsterdam Central next to the Uni Clo store'), ('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'I need to fix Kaleb Williams bicycle today What tools should i bring based on this document here https://fabster-my.sharepoint.com/:w:/g/personal/fabian_fabster_onmicrosoft_com/EfYFSS6IwSZNnyCyYPcK4IkBRiFeuXhA3cHn74Ha6Ag_Ow https://fabster-my.sharepoint.c'), ('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'how are you...'), ('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'hi adele long time'), ('19:28031fbd-

In [52]:
##Testing with GUID
search_response = chat_data[0]  # Assuming there's only one SearchResponse object in the list

# Initialize an empty list to store summaries, chat IDs, and generated GUIDs
summaries_chat_ids_and_guids = []

# Iterate over hits_containers in the SearchResponse object
for hits_container in search_response.hits_containers:
    # Iterate over each hit in the container
    for hit in hits_container.hits:
        # Extract the summary
        summary = hit.summary

        # Extract the chatId from additional_data
        chat_id = hit.resource.additional_data.get('chatId', 'Unknown')

        # Generate a unique GUID for each hit
        guid = str(uuid.uuid4())

        # Append the chatId, summary, and GUID as a tuple to the list
        summaries_chat_ids_and_guids.append((guid, summary))

# Now 'summaries_chat_ids_and_guids' contains the chatId, summary, and a unique GUID for each search hit
for item in summaries_chat_ids_and_guids:
    await kernel.memory.save_information_async("ESPC23", id=guid, text=summary)
    #print(item)

                                            

('d58f4307-d06a-46cb-8e2d-a4f4064d0232', 'hmmm which one? the one closer to the book store by Rokin near the Yo boba store?')
('8f2ebcd5-e8e2-4acc-85d6-17cf79fb4fa0', 'hey Im in Amsterdam downtown lets get some Fries from that spot near Amsterdam Central next to the Uni Clo store')
('34666169-6b2c-49be-9f77-850c9d5f4f7e', 'I need to fix Kaleb Williams bicycle today What tools should i bring based on this document here https://fabster-my.sharepoint.com/:w:/g/personal/fabian_fabster_onmicrosoft_com/EfYFSS6IwSZNnyCyYPcK4IkBRiFeuXhA3cHn74Ha6Ag_Ow https://fabster-my.sharepoint.c')
('325f8c9a-db8b-4bc9-a2b8-62a3070b0f2f', 'how are you...')
('cc65de98-596e-4a37-9e2d-c1fb2e0770e7', 'hi adele long time')
('de9a29c9-2573-4ed9-9605-2054b0e9aa97', 'u there...')
('b7ba34e9-8d7f-4773-8f1d-ec6c47a34f84', 'cool well i am going to test now on Postman...')
('8a57f16d-eda7-46a4-a300-88d5d47a89d6', 'i want to have a chat before my status change...')
('c6922b0f-ab56-4299-952e-d13fb9e4ad20', '...Hey Adele V

In [53]:
#print(summaries_chat_ids_and_guids)

In [54]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "What did i do in Amsterdam",
    ]

    for question in questions:
        print(f"Question: {question}")
        result = await kernel.memory.search_async("ESPC23", question)
        print(f"Answer: {result[0].text}\n")

In [55]:
await search_memory_examples(kernel)

Question: What did i do in Amsterdam
Answer: hi adele good morning. now my presence is open and available when you make a graph call



## Work with Drives(Files) Search Query Endpoint

In [25]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.DriveItem
			],
			query = SearchQuery(
				query_string = "Adele",
			),
		),
	],
)

drive_result = await client.search.query.post(request_body)

In [26]:
drive_data = drive_result.value

In [40]:
try:
    drive_search_response = drive_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries, from information, and generated GUIDs
    drive_summary_from_and_guids = []

    if drive_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for drive_hits_container in drive_search_response.hits_containers:
            # Iterate over each hit in the container
            for drive_hit in drive_hits_container.hits:
                # Extract the summary
                drive_summary = drive_hit.summary

                # Extract the 'from' information from additional_data
                from_data = drive_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')

                # Generate a unique GUID for each hit
                guid = str(uuid.uuid4())

                # Append the summary, 'from' information, and GUID as a tuple to the list
                drive_summary_from_and_guids.append((guid, drive_summary, from_name, from_email))
except Exception as e:
    print(f"An error occurred: {e}")

# Now 'drive_summary_from_and_guids' contains the summary, from information, and a unique GUID for each search hit
for item in drive_summary_from_and_guids:
    print(item)

('abd1fd06-aa1b-49ea-9b8f-d8dd2076f7af', '', 'Unknown', 'Unknown')
('ba535248-d6fb-4757-af0d-e9d96ffca927', 'Project Roundtree Status Report Everything is going fine. Customer paid a ton of money<ddd/>', 'Unknown', 'Unknown')
('f083ad88-2759-48d3-b3a0-b2f4a0338031', '<ddd/><c0>Adele</c0> Vance 1 856 0 {FABE1924<ddd/>', 'Unknown', 'Unknown')
('88ca038c-4e5c-46e6-94b5-37ace78045c7', '<ddd/><c0>Adele</c0> Vance 1 198 0 {5CBA3280<ddd/>', 'Unknown', 'Unknown')
('d178a91d-a56f-42d2-9f2b-868766b0d66c', ' dynamic externalitem = new JObject(); externalitem.acl = new JArray{ new JObject{{"type", "user" }, {"type", "bac58813-8954-4633-9503-cc51c0ab34c9"}, { "accessType", "grant<ddd/>', 'Unknown', 'Unknown')
('2261d084-0a71-435e-9af2-ec4007791455', '<ddd/><c0>Adele</c0> Vance 1 0 {D5FE893F-94AC<ddd/>', 'Unknown', 'Unknown')


In [41]:
print(drive_summary_from_and_guids)

[('abd1fd06-aa1b-49ea-9b8f-d8dd2076f7af', '', 'Unknown', 'Unknown'), ('ba535248-d6fb-4757-af0d-e9d96ffca927', 'Project Roundtree Status Report Everything is going fine. Customer paid a ton of money<ddd/>', 'Unknown', 'Unknown'), ('f083ad88-2759-48d3-b3a0-b2f4a0338031', '<ddd/><c0>Adele</c0> Vance 1 856 0 {FABE1924<ddd/>', 'Unknown', 'Unknown'), ('88ca038c-4e5c-46e6-94b5-37ace78045c7', '<ddd/><c0>Adele</c0> Vance 1 198 0 {5CBA3280<ddd/>', 'Unknown', 'Unknown'), ('d178a91d-a56f-42d2-9f2b-868766b0d66c', ' dynamic externalitem = new JObject(); externalitem.acl = new JArray{ new JObject{{"type", "user" }, {"type", "bac58813-8954-4633-9503-cc51c0ab34c9"}, { "accessType", "grant<ddd/>', 'Unknown', 'Unknown'), ('2261d084-0a71-435e-9af2-ec4007791455', '<ddd/><c0>Adele</c0> Vance 1 0 {D5FE893F-94AC<ddd/>', 'Unknown', 'Unknown')]


## Work with Messages(Email) Search Query Endpoint

In [29]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.Message
			],
			query = SearchQuery(
				query_string = "Adele",
			),
		),
	],
)

mail_result = await client.search.query.post(request_body)

In [30]:
mail_data = mail_result.value

In [43]:
try:
    mail_search_response = mail_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries, from information, and generated GUIDs
    mail_summary_from_and_guids = []

    if mail_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for mail_hits_container in mail_search_response.hits_containers:
            # Iterate over each hit in the container
            for mail_hit in mail_hits_container.hits:
                # Extract the summary
                mail_summary = mail_hit.summary

                # Extract the 'from' information from additional_data
                from_data = mail_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')

                # Generate a unique GUID for each hit
                guid = str(uuid.uuid4())

                # Append the summary, 'from' information, and GUID as a tuple to the list
                mail_summary_from_and_guids.append((guid, mail_summary, from_name, from_email))
except Exception as e:
    print(f"An error occurred: {e}")

# Now 'mail_summary_from_and_guids' contains the summary, from information, and a unique GUID for each search hit
for item in mail_summary_from_and_guids:
    print(item)

('24c8bcdb-7fb5-4b82-b780-bbc7dcae09d2', 'Get Outlook for iOS From: Eurostar International LTD <noreply@e.eurostar.com> Sent: Friday, November 24, 2023 1:26:32 PM To: fabian (adotob) williams <fabian@adotob.com> Subject', 'Unknown', 'Unknown')
('a0ccb465-8d40-4bc7-b4c5-c5c8528a27cb', 'Private to you   Hi, <c0>Adele</c0> Vance,  Discover trends in your work habits  An in-depth look at your work patterns in the last four weeks  Explore your insights  27 days without quiet...', 'Unknown', 'Unknown')
('c8a65f16-77d7-4e10-8db2-1b8bf74571f2', 'Private to you   Hi, <c0>Adele</c0> Vance,  Discover trends in your work habits  An in-depth look at your work patterns in the last four weeks  Work smarter with Microsoft Viva Insights  ...', 'Unknown', 'Unknown')
('b3a69919-3123-402c-a070-62abb6dfba54', "Fabian Williams shared this with you 7 days ago. If you missed it, here's a quick reminder  This link will work for anyone.  TeachKalebHowToRideABicycle  Open   Microsoft respects...", 'Unknown', 'Un

In [44]:
print(mail_summary_from_and_guids)

[('24c8bcdb-7fb5-4b82-b780-bbc7dcae09d2', 'Get Outlook for iOS From: Eurostar International LTD <noreply@e.eurostar.com> Sent: Friday, November 24, 2023 1:26:32 PM To: fabian (adotob) williams <fabian@adotob.com> Subject', 'Unknown', 'Unknown'), ('a0ccb465-8d40-4bc7-b4c5-c5c8528a27cb', 'Private to you   Hi, <c0>Adele</c0> Vance,  Discover trends in your work habits  An in-depth look at your work patterns in the last four weeks  Explore your insights  27 days without quiet...', 'Unknown', 'Unknown'), ('c8a65f16-77d7-4e10-8db2-1b8bf74571f2', 'Private to you   Hi, <c0>Adele</c0> Vance,  Discover trends in your work habits  An in-depth look at your work patterns in the last four weeks  Work smarter with Microsoft Viva Insights  ...', 'Unknown', 'Unknown'), ('b3a69919-3123-402c-a070-62abb6dfba54', "Fabian Williams shared this with you 7 days ago. If you missed it, here's a quick reminder  This link will work for anyone.  TeachKalebHowToRideABicycle  Open   Microsoft respects...", 'Unknown',

## Add Payloads to Semantic Memory